In [6]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(".."))

from src.data_eng.pipeline import main as run_pipeline

Creates a single feature (1 day return) and adds the default target (5 day horizon +1% return) on the tickers from the ticker list

In [ ]:
# first set data config

from src.config import Config

conf = Config(
    features = ['r_1d'],    # 1 day return
    target = {'horizon' : 5, 'threshold' : 0.01}, # default target
    ticker_list = ['AAPL', 'META'], # 2 tickers for example
    train_cutoff = '2019-01-01', #training end date
    validate_cutoff = '2022-01-01' #validation end date, test will be this until present
)



run_pipeline()



['AAPL', 'META']
/home/mike/Git-projects/StockPrediction/notebooks
begin fetching data from yfinance...
saved: ../data/raw/MSFT.csv
saved: ../data/raw/NVDA.csv
saved: ../data/raw/GOOGL.csv
saved: ../data/raw/AMZN.csv
saved: ../data/raw/META.csv
saved: ../data/raw/TSLA.csv
saved: ../data/raw/AVGO.csv
saved: ../data/raw/TSM.csv
saved: ../data/raw/ORCL.csv
saved: ../data/raw/WMT.csv
saved: ../data/raw/JPM.csv
saved: ../data/raw/INTC.csv
saved: ../data/raw/UNH.csv
saved: ../data/raw/HD.csv
done fetching data
being data cleaning...
done cleaning data
begin feature engineering
make features for MSFT
make features for NVDA
make features for GOOGL
make features for AMZN
make features for META
make features for TSLA
make features for AVGO
make features for TSM
make features for ORCL
make features for WMT
make features for JPM
make features for INTC
make features for UNH
make features for HD
Done egineering features


KeyError: 'AAPL'

Read the Data back in after processing to a data bundle. This pre splits our data into Validation